In [126]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [127]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [138]:
#https://drive.google.com/file/d/1Cj_gZFUmvegKICLQxSuPRk0QT5zvtPgK/view?usp=drive_link
#https://drive.google.com/file/d/1HObEZNhDxOCCFcVnRQzW9y3w4nd_yiDx/view?usp=drive_link
#https://drive.google.com/file/d/1ovu6KTPb8bc0h06A2nvy-kahAmecQjhy/view?usp=drive_link
your_file_all = drive.CreateFile({'id':'1Cj_gZFUmvegKICLQxSuPRk0QT5zvtPgK'})
your_file_ta = drive.CreateFile({'id':'1HObEZNhDxOCCFcVnRQzW9y3w4nd_yiDx'})
your_file_oc = drive.CreateFile({'id':'1ovu6KTPb8bc0h06A2nvy-kahAmecQjhy'})

In [140]:
your_file_all.GetContentFile('bitcoin-all-on-chain-and-technical-indicators.csv')
your_file_ta.GetContentFile('bitcoin-all-technical-indicators.csv')
your_file_oc.GetContentFile('bitcoin-all-on-chain.csv')

In [130]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import os
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.utils import to_categorical

In [141]:
ta_df = pd.read_csv('bitcoin-all-technical-indicators.csv')
ta_df['timestamp'] = pd.to_datetime(ta_df['timestamp'])
ta_df.set_index('timestamp', inplace=True)
#ta_df = ta_df.dropna()
ta_df_returns = ta_df.pct_change()
# Replace infinities with NaN
ta_df_returns = ta_df_returns.replace([np.inf, -np.inf], np.nan)
ta_df_returns.fillna(0, inplace=True)
ta_df = add_target_to_dataframe(ta_df)

oc_df = pd.read_csv('bitcoin-all-on-chain.csv')
oc_df['timestamp'] = pd.to_datetime(oc_df['timestamp'])
oc_df.set_index('timestamp', inplace=True)
oc_df_returns = oc_df.pct_change()
oc_df_returns = oc_df_returns.replace([np.inf, -np.inf], np.nan)
oc_df_returns.fillna(0, inplace=True)
oc_df = add_target_to_dataframe(oc_df)

oc_ta_df = pd.read_csv('bitcoin-all-on-chain-and-technical-indicators.csv')
oc_ta_df['timestamp'] = pd.to_datetime(oc_ta_df['timestamp'])
oc_ta_df.set_index('timestamp', inplace=True)
oc_ta_df_returns = oc_ta_df.pct_change()
oc_ta_df_returns = oc_ta_df_returns.replace([np.inf, -np.inf], np.nan)
oc_ta_df_returns.fillna(0, inplace=True)
oc_ta_df = add_target_to_dataframe(oc_ta_df)

price_only_df = ta_df[['price']]
price_only_df_returns = price_only_df.pct_change()
price_only_df_returns = price_only_df_returns.dropna()
price_only_df = add_target_to_dataframe(price_only_df)

increase     1553
decrease     1350
no change      16
Name: target, dtype: int64
increase     1553
decrease     1350
no change      16
Name: target, dtype: int64
increase     1553
decrease     1350
no change      16
Name: target, dtype: int64
increase     1553
decrease     1349
no change      16
Name: target, dtype: int64


<ipython-input-123-9fe87a8b200c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'price_change'] = df['price'].pct_change()
<ipython-input-123-9fe87a8b200c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-123-9fe87a8b200c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

In [123]:
def add_target_to_dataframe(df, threshold=0.0001):
    # Compute the percentage change from the current to the next day's price
    df.loc[:, 'price_change'] = df['price'].pct_change()
    df.dropna(inplace=True)

    # Define the target variable based on the price change
    df.loc[:, 'target'] = 'no change'
    df.loc[df['price_change'] > threshold, 'target'] = 'increase'
    df.loc[df['price_change'] < -threshold, 'target'] = 'decrease'

    print(df['target'].value_counts())


    return df

In [7]:
def plot_values(timestamp, *args):
    import matplotlib

    if len(args) % 2 != 0:
        raise ValueError("Every feature should have a corresponding name")

    # Use a predefined style
    plt.style.use('ggplot')

    # Set figure size
    fig, ax = plt.subplots(figsize=(12,6))

    # Create a colormap that will generate colors
    colormap = matplotlib.colormaps['tab10']

    # Plot data with customized line for each feature
    for i in range(0, len(args), 2):
        feature_name = args[i]
        feature_values = args[i+1]
        color = colormap(i // 2 / (len(args)//2))
        ax.plot(timestamp, feature_values, label=feature_name, color=color, linewidth=1)

    # Set labels with improved readability
    ax.set_xlabel('Date')
    ax.yaxis.set_label_position("right") # This line moves y label to the right
    ax.set_ylabel('Feature Values')
    ax.yaxis.tick_right() # This line moves y axis to the right
    ax.set_title('Features over Time')

    # Format the timestamps and set locator
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # format as year-month-day
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3)) # display a label at the start of every 3rd month

    # Rotate x-axis labels for better visibility
    plt.xticks(rotation=45)

    # Add a grid
    ax.grid(True)

    # Add legend
    ax.legend()

    # Show plot
    plt.show()

In [67]:
from sklearn.preprocessing import OneHotEncoder


def data_preparation_classifier(df, lookback, future, scale, test_size=0.1):
    # Convert 'Date' column to datetime
    date_train = pd.to_datetime(df.index)

    target_column = 'target'  # assuming your target column is named 'target'

    # Separate features and target
    features = df.drop(target_column, axis=1)
    target = df[[target_column]]

    # Split data into training and testing before scaling
    df_features_train, df_features_test, target_train, target_test = train_test_split(features, target, test_size=test_size, shuffle=False)

    # Scale features
    df_train_scaled = scale.fit_transform(df_features_train)
    df_test_scaled = scale.transform(df_features_test)  # use the scaler fitted on the training data

    # One-hot encode target variable
    encoder = OneHotEncoder(sparse=False)
    target_train_encoded = encoder.fit_transform(target_train)
    target_test_encoded = encoder.transform(target_test)

    # Create the feature and target arrays
    X_train, y_train = [], []
    for i in range(lookback, len(df_train_scaled)-future+1):
        X_train.append(df_train_scaled[i-lookback:i, :])
        y_train.append(target_train_encoded[i+future-1])

    X_test, y_test = [], []
    for i in range(lookback, len(df_test_scaled)-future+1):
        X_test.append(df_test_scaled[i-lookback:i, :])
        y_test.append(target_test_encoded[i+future-1])

    # Convert to numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")
    return X_train, X_test, y_train, y_test, date_train, encoder



In [108]:
def lstm_classifier(X, y):
    model_name="lstm_1_"+str(int(time.time()))
    model=Sequential(name=model_name)

    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    model.add(Dense(25))
    model.add(Dense(3, activation='softmax'))

    adam = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])  # And this

    model.fit(X, y, validation_split=0.1, epochs=50, batch_size=32)
    return model

In [9]:
def lstm_1(X, y):
    model_name="lstm_1_"+str(int(time.time()))
    model=Sequential(name=model_name)

    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    model.add(Dense(25))
    model.add(Dense(1))

    adam = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error',optimizer=adam)

    model.fit(X, y, validation_split=0.1, epochs=50, batch_size=32)
    return model

In [10]:
def lstm_2(X, y):
    model_name="lstm_1_"+str(int(time.time()))
    model=Sequential(name=model_name)

    model.add(LSTM(units=256, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=128, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    adam = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error',optimizer=adam)

    model.fit(X, y, validation_split=0.1, epochs=50, batch_size=32)
    return model

In [50]:
from keras.regularizers import l2

def lstm_3(X, y):
    model_name="lstm_3_"+str(int(time.time()))
    model=Sequential(name=model_name)

    model.add(LSTM(512, return_sequences=True, input_shape=(X.shape[1], X.shape[2]), kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(3, activation='softmax'))  # Change this

    adam = optimizers.Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])  # And this

    model.fit(X, y, validation_split=0.1, epochs=100, batch_size=32, verbose=1)

    return model


In [75]:
def predict_classifier(df):
    scale = MinMaxScaler(feature_range=(0,1))
    X_train, X_test, y_train, y_test, date_train, encoder = data_preparation_classifier(df, 60, 1, scale)
    print(y_train)

    model = lstm_classifier(X_train, y_train)
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    predicted_classes_onehot = to_categorical(predicted_classes, num_classes=y_train.shape[1])


    # Inverse transform the predicted classes back to original labels
    predicted_labels = encoder.inverse_transform(predicted_classes_onehot)
    print(predicted_labels)

    # Calculate accuracy
    accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes)
    print('Model Accuracy:', accuracy)

In [142]:
predict_classifier(ta_df)
predict_classifier(oc_df)
predict_classifier(oc_ta_df)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


X_train shape: (2567, 60, 89)
y_train shape: (2567, 3)
X_test shape: (232, 60, 89)
y_test shape: (232, 3)
[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
Epoch 1/50
73/73 [==============================] - 10s 35ms/step - loss: 0.7452 - accuracy: 0.5208 - val_loss: 0.7645 - val_accuracy: 0.4786
Epoch 2/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7208 - accuracy: 0.5281 - val_loss: 0.7438 - val_accuracy: 0.4786
Epoch 3/50
73/73 [==============================] - 1s 18ms/step - loss: 0.7215 - accuracy: 0.5377 - val_loss: 0.7452 - val_accuracy: 0.5136
Epoch 4/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7225 - accuracy: 0.5238 - val_loss: 0.7423 - val_accuracy: 0.5097
Epoch 5/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7187 - accuracy: 0.5398 - val_loss: 0.7477 - val_accuracy: 0.5136
Epoch 6/50
73/73 [==============================] - 2s 24ms/step - loss: 0.7157 - accuracy: 0.5290 - val_loss:

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
73/73 [==============================] - 11s 39ms/step - loss: 0.7480 - accuracy: 0.5234 - val_loss: 0.7420 - val_accuracy: 0.4786
Epoch 2/50
73/73 [==============================] - 2s 25ms/step - loss: 0.7218 - accuracy: 0.5199 - val_loss: 0.7629 - val_accuracy: 0.5136
Epoch 3/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7181 - accuracy: 0.5511 - val_loss: 0.7484 - val_accuracy: 0.5136
Epoch 4/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7164 - accuracy: 0.5364 - val_loss: 0.7387 - val_accuracy: 0.5136
Epoch 5/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7166 - accuracy: 0.5381 - val_loss: 0.7380 - val_accuracy: 0.4825
Epoch 6/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7100 - accuracy: 0.5407 - val_loss: 0.7445 - val_accuracy: 0.4825
Epoch 7/50
73/73 [==============================] - 1s 17ms/step - loss: 0.7148 - accuracy: 0.5385 - val_loss: 0.7412 - val_accuracy: 0.4942
Epoch 8/50
7

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
73/73 [==============================] - 12s 40ms/step - loss: 0.7473 - accuracy: 0.5199 - val_loss: 0.7436 - val_accuracy: 0.4786
Epoch 2/50
73/73 [==============================] - 1s 19ms/step - loss: 0.7223 - accuracy: 0.5398 - val_loss: 0.7658 - val_accuracy: 0.5136
Epoch 3/50
73/73 [==============================] - 1s 18ms/step - loss: 0.7191 - accuracy: 0.5372 - val_loss: 0.7427 - val_accuracy: 0.5136
Epoch 4/50
73/73 [==============================] - 1s 18ms/step - loss: 0.7142 - accuracy: 0.5550 - val_loss: 0.7448 - val_accuracy: 0.5136
Epoch 5/50
73/73 [==============================] - 1s 19ms/step - loss: 0.7133 - accuracy: 0.5489 - val_loss: 0.7419 - val_accuracy: 0.4747
Epoch 6/50
73/73 [==============================] - 1s 20ms/step - loss: 0.7135 - accuracy: 0.5416 - val_loss: 0.7419 - val_accuracy: 0.5136
Epoch 7/50
73/73 [==============================] - 2s 29ms/step - loss: 0.7127 - accuracy: 0.5442 - val_loss: 0.7342 - val_accuracy: 0.5292
Epoch 8/50
7